# YOLO na Prática: Inferência

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

## 1. Carregando o Modelo Pré-Treinado

In [ ]:
# Peso padrão do modelo: YOLOv11 treinado no COCO
model = YOLO()

## 2. Realizando a Inferência

In [ ]:
image_path = 'frame1.jpg'
img = cv2.imread(image_path)

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.title('Imagem de Entrada')
plt.axis('off')
plt.show()

In [ ]:
results = model.predict(source=image_path, conf=0.25, iou=0.7, verbose=False)
print(f"Inferência concluída. Foram encontrados {len(results[0])} objeto(s) detectado(s) na imagem.")
# results é uma lista (mesmo para uma única imagem, pois pode-se passar uma lista de imagens)
# Cada elemento da lista é um objeto 'Results'.

## 3. Analisando o resultado da YOLO

In [ ]:
result = results[0]

# result.boxes: Contém as bounding boxes, classes, confianças.
boxes = result.boxes
print(f"Tipo de objeto 'boxes': {type(boxes)}")
print(f"Número de detecções: {len(boxes)}")

print("-"*80)
# Exibindo os dados brutos das detecções
for detection in boxes:
    print(detection.data.cpu().numpy().astype(float))
    print("-"*80)

In [ ]:
for detection in boxes:
    print(f"Coordenadas: {detection.xyxy.cpu().numpy()}") # x1, y1, x2, y2
    print(f"Confiança: {detection.conf.cpu().numpy()}")
    print(f"ID da Classe: {detection.cls.cpu().numpy()}")
    print(f"Nome da Classe: {model.names[detection.cls.cpu().numpy().astype(int)[0]]}")
    print("-"*80)

In [ ]:
plotted_image = result.plot() # result.plot() retorna um array NumPy (RGB)

plotted_image = cv2.cvtColor(plotted_image, cv2.COLOR_BGR2RGB)  # Converte de BGR para RGB para exibição correta com matplotlib
plt.imshow(plotted_image)
plt.title('Detecções (Plotadas pelo Ultralytics)')
plt.axis('off')
plt.show()

In [ ]:
img_plot = img.copy()

# Acessando os nomes das classes diretamente do modelo
class_names = model.names

# Loopar por cada detecção
for box in boxes:
    # Coordenadas da bounding box (formato xyxy: x_min, y_min, x_max, y_max)
    x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())

    # Confiança da detecção
    confidence = float(box.conf[0].cpu().numpy())

    # ID da classe
    class_id = int(box.cls[0].cpu().numpy())
    class_name = class_names[class_id]

    # Filtro de confiança (ex: só mostrar se confiança > 0.5)
    if confidence > 0.5:

        # Desenhar o retângulo
        color = (0, 255, 0) # Verde para BGR
        thickness = 10
        cv2.rectangle(img_plot, (x1, y1), (x2, y2), color, thickness)

        # Colocar o label (nome da classe + confiança)
        label = f'{class_name} {confidence:.2f}'
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_thickness = 2
        text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_size[1] + 10 # Posição do texto
        
        cv2.rectangle(img_plot, (text_x, text_y - text_size[1] - 5), (text_x + text_size[0] + 5, text_y + 5), color, -1) # Fundo do texto
        cv2.putText(img_plot, label, (text_x, text_y), font, font_scale, (0, 0, 0), font_thickness, cv2.LINE_AA) # Texto preto

# Exibir a imagem com as detecções plotadas manualmente
img_plot_rgb = cv2.cvtColor(img_plot, cv2.COLOR_BGR2RGB)
plt.imshow(img_plot_rgb)
plt.title('Detecções (Plotadas Manualmente)')
plt.axis('off')
plt.show()

# 4. Realizando inferência em videos

In [ ]:
video_path = 'vehicle-counting.mp4'
output_video_path = 'output_video_yolo.mp4'

cap = cv2.VideoCapture(video_path)

# Propriedades do vídeo
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Resolução do vídeo: {width}x{height}, FPS: {fps}")

# Configurar o VideoWriter para salvar o vídeo com detecções
# 'mp4v' é um codec comum para arquivos .mp4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

In [ ]:
frame_count = 0
while True:
        ret, frame = cap.read()
        if not ret:
            break # Fim do vídeo ou erro na leitura

        results = model.predict(source=frame, conf=0.25, iou=0.7, verbose=False)

        annotated_frame = results[0].plot() # results[0] pois processamos um frame por vez

        cv2.imshow('Deteccao YOLO', annotated_frame)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break # Pressione 'q' para sair

        out.write(annotated_frame)

        frame_count += 1

In [ ]:
print(f"\nTotal de frames processados: {frame_count}")
cap.release() # Liberar o objeto de captura
out.release() # Liberar o objeto de escrita de vídeo
cv2.destroyAllWindows() # Fechar todas as janelas
print(f"Vídeo com detecções salvo como '{output_video_path}'")

## 4.1 Melhorando a inferência em vídeo

In [ ]:
video_path = 'vehicle-counting.mp4'
output_video_path = 'output_video_yolo.mp4'

cap = cv2.VideoCapture(video_path)

# Propriedades do vídeo
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Resolução do vídeo: {width}x{height}, FPS: {fps}")

# Configurar o VideoWriter para salvar o vídeo com detecções
# 'mp4v' é um codec comum para arquivos .mp4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

In [ ]:
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break # Fim do vídeo ou erro na leitura

    results = model.predict(source=frame, conf=0.25, iou=0.7, verbose=False)
    
    # Acessando os nomes das classes diretamente do modelo
    class_names = model.names

    boxes = results[0].boxes  # Obter as detecções
    # Loopar por cada detecção
    for box in boxes:
        # Coordenadas da bounding box (formato xyxy: x_min, y_min, x_max, y_max)
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())

        # Confiança da detecção
        confidence = float(box.conf[0].cpu().numpy())

        # ID da classe
        class_id = int(box.cls[0].cpu().numpy())
        class_name = class_names[class_id]

        # Filtro de confiança (ex: só mostrar se confiança > 0.5)
        if confidence > 0.5:

            # Desenhar o retângulo
            color = (0, 255, 0) # Verde para BGR
            thickness = 10
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            # Colocar o label (nome da classe + confiança)
            label = f'{class_name} {confidence:.2f}'
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 2
            font_thickness = 2
            text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
            text_x = x1
            text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_size[1] + 10 # Posição do texto
            
            cv2.rectangle(frame, (text_x, text_y - text_size[1] - 5), (text_x + text_size[0] + 5, text_y + 5), color, -1) # Fundo do texto
            cv2.putText(frame, label, (text_x, text_y), font, font_scale, (0, 0, 0), font_thickness, cv2.LINE_AA) # Texto preto

    cv2.imshow('Deteccao YOLO', frame)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break # Pressione 'q' para sair
    
    out.write(annotated_frame)
    frame_count += 1

In [ ]:
print(f"\nTotal de frames processados: {frame_count}")
cap.release() # Liberar o objeto de captura
out.release() # Liberar o objeto de escrita de vídeo
cv2.destroyAllWindows() # Fechar todas as janelas
print(f"Vídeo com detecções salvo como '{output_video_path}'")